This is an introductory lecture designed to introduce people from outside of Computer Vision to the Image Classification problem, and the data-driven approach. The Table of Contents:

* Intro to Image Classification, data-driven approach, pipeline
* Nearest Neighbor Classifier
* k-Nearest Neighbor
* Validation sets, Cross-validation, hyperparameter tuning
* Pros/Cons of Nearest Neighbor
* Summary
* Summary: Applying kNN in practice
* Further Reading

Suppose now that we are given the CIFAR-10 training set of 50,000 images (5,000 images for every one of the labels), and we wish to label the remaining 10,000. The nearest neighbor classifier will take a test image, compare it to every single one of the training images, and predict the label of the closest training image. In CIFAR-10, the images are 32 pixels high and wide. Each image is labeled with one of 10 classes (for example 'airplane, automobole, bird, etc;).

One of the simplest possibilites is to compare the images pixel by pixel and add up all the differences. In other words, given two images and representing them as vectors I1, I2, a resonable choice for comparing them might be the L1 distance:

d1(I1,I2)=∑p|Ip1−Ip2|

<img src='http://cs231n.github.io/assets/nneg.jpeg'>

In [ ]:
import numpy as np
import pickle
import sys
from sklearn.neighbors import  KNeighborsClassifier
from tqdm import *
import operator

In [ ]:
def load_CIFAR10_detail(filename):
    
    with open(filename, 'rb') as f:
        dict = pickle.load(f, encoding='latin1')
        
        x = dict['data']
        y = dict['labels']
        x = x.astype(float)
        y = np.array(y)
        
    return x,y


def load_CIFAR10(foldername):
    
    xs = []
    ys = []
    for i in range(1,6):
        file = foldername+ '/data_batch_' + str(i)
        X, Y = load_CIFAR10_detail(file)
        xs.append(X)
        ys.append(Y)
    
    x_train = np.concatenate(xs)
    y_train = np.concatenate(ys)
    
    del xs, ys
    
    x_test, y_test = load_CIFAR10_detail('cifar-10-batches-py/test_batch')
    
    return x_train, y_train, x_test, y_test

In [ ]:
Xtr, Ytr, Xte, Yte = load_CIFAR10('cifar-10-batches-py')

In [ ]:
# http://machinelearningmastery.com/tutorial-to-implement-k-nearest-neighbors-in-python-from-scratch/

class NearestNeighbor(object):
    def __init__(self):
        pass
    
    def train(self, X, y):
        """ X is N x D where each row is an example. Y is 1-dimension of size N """
        # the nearest neighbor classifier simply remembers all the training data
        self.Xtr = X
        self.ytr = y
        
    def predict(self, X,distance='L1',k=1):
        """ X is N x D where each row is an example we wish to predict label for """
        num_test = X.shape[0]
        Ypred = np.zeros(num_test, dtype=self.ytr.dtype)
        for i in tqdm(range(num_test)):
            if distance == 'L1':
                distances = np.sum(np.abs(self.Xtr - X[i,:]), axis = 1)
            elif distance == 'L2':
                distances = np.sqrt(np.sum(np.square(self.Xtr - X[i,:]), axis=1))
            # sort the array and get the index/indices based on k value
            sort_neighbors = np.ndarray.argsort(distances)
            k_neighbors = sort_neighbors[-k:]
            
            # votes the best classes based on number of occurences
            classVotes = {}
            for index in k_neighbors:
                pred = self.ytr[index]
                if pred in classVotes:
                    classVotes[pred] += 1
                else:
                    classVotes[pred] = 1
            
            # sort based on number of occurences
            sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
            
            # get the first class sorted
            Ypred[i] = self.ytr[sortedVotes[0][0]]
        
        return Ypred

In [ ]:
nn = NearestNeighbor()
nn.train(Xtr, Ytr)
Yte_predict = nn.predict(Xte,distance='L2', k=10)
print('accuracy; {.2f}'.format(np.mean(Yte_predict == Yte)))